In [1]:
# with open("iris.data")as f:
#     data = f.read()
#     data = data.split("\n")
# newData = []
# for line in data:
# #     print(line)
#     newData.append(line.split(","))
# # print(newData)
# df = pd.DataFrame(newData, columns=["T1","T2","T3","T4","Type"])
# df.to_csv("py.csv",index=False)

In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Undersatanding Data

In [4]:
print("The shape of train set:",df_train.shape)
print("The Shape of test set:",df_test.shape)

The shape of train set: (1460, 81)
The Shape of test set: (1459, 80)


# Data Preprocessing

In [5]:
df_train.set_index("Id", inplace=True)
df_test.set_index("Id", inplace=True)
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Handeling Missing Data

In [6]:
df_train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [7]:
cols_with_null = df_train.isnull().sum().sort_values(ascending=False)
cols_with_null.head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtCond          37
BsmtQual          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
MSSubClass         0
dtype: int64

In [8]:
print("Total number of missing data in the dataset:",df_train.isnull().sum().sum())

Total number of missing data in the dataset: 6965


In [9]:
df_train["SalePrice"].isnull().sum()

0

In [10]:
# cols_to_drop = (cols_with_null.head(6).index).tolist()
# df_train.drop(cols_to_drop,axis=1,inplace=True)
# df_test.drop(cols_to_drop,axis=1,inplace=True)

# Creating the target variable

In [11]:
y = df_train.SalePrice
X = df_train.drop(["SalePrice"],axis=1)

# Splitting the data

In [12]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.8,random_state=0)

# Handling the categorical and numerical columns

In [13]:
categorical_cols=[cname for cname in X_train.columns
                 if X_train[cname].nunique()<10 and
                 X_train[cname].dtype=="object"]

In [14]:
numerical_cols=[cname for cname in X_train.columns
                 if X_train[cname].dtype in ["int64","float64"]]

In [15]:
print("the number of categorical columns:",len(categorical_cols))
print("the number of numerical columns:",len(numerical_cols))

the number of categorical columns: 40
the number of numerical columns: 36


In [16]:
my_cols=categorical_cols+numerical_cols
X_train = X_train[my_cols]
X_val = X_val[my_cols]
X_test = df_test[my_cols]

# Pipeline for data preprocessing

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [18]:
numerical_transformer = Pipeline(steps=[("imputer_num",SimpleImputer(strategy="median")),
                                        ("scaler",StandardScaler())])

In [19]:
categorical_transformer = Pipeline(steps=[("imputer_cal",SimpleImputer(strategy="most_frequent")),
                                        ("onehot",OneHotEncoder(handle_unknown="ignore"))])

In [20]:
preprocessor = ColumnTransformer(transformers=[("num",
                                                numerical_transformer,numerical_cols),
                                               ("cat",categorical_transformer,categorical_cols)])

# Model Building

In [21]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)

In [22]:
my_pipeline = Pipeline(steps=[ ("preprocessor", preprocessor),
                              ("model", rf)])

In [23]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Low...
                                                   'LotConfig', 'LandSlope',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'MasVnrType', 'ExterQual',
                                                   'ExterCond', 'Foundation',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...])])),
                ('model', RandomForestRegressor(random_state=0))])

# Model evaluation

In [24]:
val_preds = my_pipeline.predict(X_val)

In [25]:
from sklearn.metrics import mean_absolute_error
print("Validation MAE:",mean_absolute_error(y_val,val_preds))

Validation MAE: 17584.598458904107


# Cross Validation

In [26]:
from sklearn.model_selection import cross_val_score
scores = -1*cross_val_score(my_pipeline, X,y, cv=5,
                          scoring="neg_mean_absolute_error")
print("mean Cross Validation Score:",scores.mean())

mean Cross Validation Score: 17682.311315068495


# Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'model__n_estimators': [500, 600, 700],
    'model__max_features': ['auto','sqrt','log2'],
    'model__max_depth' : [5,6,7],
    'model__criterion' :['squared_error','absolute_error','poisson']}
GridCV = GridSearchCV(my_pipeline, param_grid, n_jobs= -1)
GridCV.fit(X_train,y_train)  
print(GridCV.best_params_)    
print(GridCV.best_score_)

In [27]:
# preds_test = GridCV.predict(X_test)